# Import libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sympy as sy
import keras
import os
from matplotlib import pyplot as plt
import alpaca_trade_api as tradeapi
import threading
import alpaca_trade_api as alpaca
import time
import datetime
import pickle as pk

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
api_secret='Uo/Kjc6/3kh3sR6aN30u01Yw0IXnW/w1Yp2Hyp2O'
api_key='PKUD6ZVHED4O8256YKR6'
api = tradeapi.REST(api_key, api_secret, 'https://paper-api.alpaca.markets','v2') # or use ENV Vars shown below
account = api.get_account()
api.list_positions()

[Position({   'asset_class': 'us_equity',
     'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
     'avg_entry_price': '674.985',
     'change_today': '-0.0044728690450758',
     'cost_basis': '1349.97',
     'current_price': '745.61',
     'exchange': 'NASDAQ',
     'lastday_price': '748.96',
     'market_value': '1491.22',
     'qty': '2',
     'side': 'long',
     'symbol': 'TSLA',
     'unrealized_intraday_pl': '-6.7',
     'unrealized_intraday_plpc': '-0.0044728690450758',
     'unrealized_pl': '141.25',
     'unrealized_plpc': '0.1046319547841804'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': 'f30d734c-2806-4d0d-b145-f9fade61432b',
     'avg_entry_price': '1464.52',
     'change_today': '0.0020322036539022',
     'cost_basis': '2929.04',
     'current_price': '1479.23',
     'exchange': 'NASDAQ',
     'lastday_price': '1476.23',
     'market_value': '2958.46',
     'qty': '2',
     'side': 'long',
     'symbol': 'GOOG',
     'unrealized_intraday_pl': '6',
    

In [3]:
stockUniverse = ['GOOG','DOMO', 'TLRY', 'SQ', 'MRO', 'AAPL', 'GM', 'SNAP', 'SHOP', 'SPLK', 'BA', 'AMZN', 'SUI', 'SUN', 'TSLA', 'CGC', 'SPWR', 'NIO', 'CAT', 'MSFT', 'PANW', 'OKTA', 'TWTR', 'TM', 'RTN', 'ATVI', 'GS', 'BAC', 'MS', 'TWLO', 'QCOM', ]
# Format the allStocks variable for use in the class.
allStocks = []
for stock in stockUniverse:
  allStocks.append([stock, 0])

In [4]:
n=5

In [5]:
bars=api.get_barset(','.join(stockUniverse),
                     '5Min',limit=100)

In [6]:
bars['GOOG'][99].t

Timestamp('2020-02-07 15:55:00-0500', tz='America/New_York')

In [7]:
class My_df:
    def __init__():
        pass
    def s_bars_to_dfdic(bars):
        symbols=list(bars.keys())
        dfdic={}
        columns=['o','h','l','c','v','t']
        for symbol in symbols:
            df=pd.DataFrame(columns=columns)
            for step in bars[symbol]:
                temp_df=pd.DataFrame(columns=columns,data=[[step.o,
                                                           step.h,
                                                           step.l,
                                                           step.c,step.v,step.t.isoformat()]])
                df=df.append(temp_df,ignore_index=True)
            dfdic[symbol]=df
        return dfdic
    def bars_to_dfdic(self,bars):
        symbols=list(bars.keys())
        dfdic={}
        columns=['o','h','l','c','v','t']
        for symbol in symbols:
            df=pd.DataFrame(columns=columns)
            for step in bars[symbol]:
                temp_df=pd.DataFrame(columns=columns,data=[[step.o,
                                                           step.h,
                                                           step.l,
                                                           step.c,step.v,step.t.isoformat()]])
                df=df.append(temp_df,ignore_index=True)
            dfdic[symbol]=df
        self.dfdic= dfdic
    def save_to_pkl(self,file='data.pkl'):
        with open(file,'wb') as f:
            pk.dump(obj=self.dfdic,file=f)
    def load_from_file(file='data.pkl'):
        with open(file,'rb') as f:
            dfdic=pk.load(f)
        return dfdic
    def merge(dfdic1,dfdic2,symbols):
        dfdic={}
        for df1,df2,symbol in zip(dfdic1.values(),dfdic2.values(),symbols):
            df1=df1.append(df2,ignore_index=True).drop_duplicates()
            dfdic[symbol]=df1
        return dfdic

In [8]:
mdf=My_df
mdf.bars_to_dfdic(mdf,
                  bars=bars)

In [9]:
dfdic1=My_df.load_from_file()

In [10]:
My_df.merge(dfdic1,mdf.dfdic,list(mdf.dfdic.keys()))

{'AAPL':            o        h        l        c      v                          t
 0    313.680  313.790  313.680  313.710   1017  2020-01-16T14:10:00-05:00
 1    313.815  314.020  313.815  313.910   1339  2020-01-16T14:15:00-05:00
 2    313.930  313.990  313.785  313.790   1287  2020-01-16T14:20:00-05:00
 3    313.720  313.925  313.720  313.910    539  2020-01-16T14:25:00-05:00
 4    313.880  313.965  313.810  313.965    763  2020-01-16T14:30:00-05:00
 ..       ...      ...      ...      ...    ...                        ...
 195  319.165  319.225  318.945  319.060   3201  2020-02-07T15:35:00-05:00
 196  319.025  319.855  318.995  319.540   8071  2020-02-07T15:40:00-05:00
 197  319.635  319.820  319.485  319.570  13415  2020-02-07T15:45:00-05:00
 198  319.820  319.845  319.490  319.610   4400  2020-02-07T15:50:00-05:00
 199  319.680  320.030  319.680  320.030  10338  2020-02-07T15:55:00-05:00
 
 [200 rows x 6 columns],
 'AMZN':             o         h         l         c     v       